This notebook analyzes the Titanic passenger dataset to identify factors influencing survival outcomes. The focus is on exploratory data analysis and data preprocessing on the Titanic dataset to identify survival patterns and prepare a clean dataset for machine learning modeling.

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load Dataset into Dataframe

In [ ]:
df=pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')

# Exploratory Data Analysis

## 1. Dataset Overview

In [ ]:
df.head()  #head views the first five rows of the dataframe

In [ ]:
df.info()

## 2. Target Variable Distribution

In [ ]:
sns.countplot(x="Survived", data=df)
plt.title("Survival Distribution")
plt.show()

## 3.Missing Value Analysis

In [ ]:
missing = df.isnull().sum()
print(missing)

## 4. Survival by Gender

In [ ]:
sns.countplot(x="Sex", hue="Survived", data=df)
plt.title("Survival by Gender")
plt.show()

### Inference:<br> 
Female passengers show a significantly higher survival rate compared to males,indicating that gender was a strong determinant of survival.

## 5. Survival by Passenger Class

In [ ]:
sns.countplot(x="Pclass", hue="Survived", data=df)
plt.title("Survival by Passenger Class")
plt.show()

### Inference: <br>
Passengers traveling in higher classes (Pclass = 1) had better survival rates,suggesting socio-economic status influenced access to lifeboats.

## 6. Age Distribution

In [ ]:
sns.histplot(df["Age"], bins=30, kde=True)
plt.title("Age Distribution")
plt.show()

### Inference: <br>
The age distribution is right-skewed, with a higher concentration of passengers between 20 and 40 years. Children appear to have relatively higher survival chances.

## 7. Fare Vs Survival

In [ ]:
sns.boxplot(x="Survived", y="Fare", data=df)
plt.title("Fare vs Survival")
plt.show()

### Inference: <br>
Survivors generally paid higher fares, reinforcing the relationship between passenger class, fare, and survival probability.

## 8. Data Cleaning

### 1. Drop columns with high cardinality or limited predictive value

In [ ]:
df.drop(columns=["Cabin", "Name", "PassengerId"], inplace=True)

### 2. Fill missing Age with median

In [ ]:
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

## 9.Encoding Categorical Variables

In [ ]:
# Binary encoding for Sex
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})


# One-hot encoding for Embarked
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

## 10. Final Dataset Overview

In [ ]:
print(df.head())
print(df.info())

In [ ]:
# Save Cleaned Dataset
df.to_csv("titanic_cleaned.csv", index=False)
print("Cleaned dataset saved as titanic_cleaned.csv")

# Key Insights from EDA

- Gender is a strong predictor of survival, with females having higher survival rates. <br>
- Passenger class and fare are positively correlated with survival probability. <br>
- Younger passengers, particularly children, show better survival outcomes. <br>
- Missing values in Age and Embarked must be handled carefully to avoid bias. 

This notebook provides a structured exploratory analysis of the Titanic dataset and prepares a clean dataset suitable for machine learning models. The insights derived from EDA guide feature selection and engineering in the modeling phase.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
#Load Data
df=pd.read_csv("titanic_cleaned.csv")

In [ ]:
# Target and features
X = df.drop("Survived", axis=1)
y = df["Survived"]

# Feature Engineering

In [ ]:
# Feature engineering
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

# Ticket-based features (make sure Ticket exists in df)
ticket_counts = df["Ticket"].value_counts()
df["TicketFreq"] = df["Ticket"].map(ticket_counts)
df["TicketPrefix"] = df["Ticket"].str.extract('([A-Za-z]+)', expand=False).fillna("None")

# Drop original Ticket after features
df = df.drop(columns=["Ticket"])

# Separate features and target
X = df.drop("Survived", axis=1)  # X contains all input features
y = df["Survived"]               # y contains the target


In [ ]:
#Feature Groups
numeric_features = ["Age", "Fare", "FamilySize", "TicketFreq"]
categorical_features = ["Pclass", "Sex", "Embarked", "IsAlone","TicketPrefix"]

# Preprocessing Pipeline

In [ ]:
from sklearn.preprocessing import OneHotEncoder

numeric_features = ["Age", "Fare", "FamilySize", "TicketFreq"]
categorical_features = ["Pclass", "Sex", "Embarked", "IsAlone"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"),
         categorical_features)
    ]
)

# Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Model- Logistic Regression

In [ ]:
log_reg = Pipeline(steps=[("preprocessor", preprocessor),("classifier", LogisticRegression(max_iter=1000))])
log_reg.fit(X_train, y_train)

# Predictions
y_pred_lr = log_reg.predict(X_test)

print("Logistic Regression Results")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

# Model - Random Forest

In [ ]:
rf = Pipeline(steps=[("preprocessor", preprocessor),
                     ("classifier", RandomForestClassifier(random_state=42))])
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Results")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Hyperparameter Tuning - Random Forest

In [ ]:
param_grid = {"classifier__n_estimators": [100, 200],
              "classifier__max_depth": [None, 5, 10],
              "classifier__min_samples_split": [2, 5]}

grid_search = GridSearchCV(rf,param_grid,cv=5,scoring="accuracy",n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Final Evaluation

In [ ]:
y_pred_best = best_model.predict(X_test)

print("Tuned Random Forest Results")
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best))

# Model - Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

gb = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", GradientBoostingClassifier(random_state=42))
])

gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(best_model, X, y, cv=5)
print(scores.mean())

# Results and Conclusion

## 1. Model Performance Comparison

| Model                  | Accuracy                | Notes                                                                            |
| ---------------------- | ----------------------- | -------------------------------------------------------------------------------- |
| Logistic Regression    | 0.8100                  | Baseline linear model; interpretable coefficients                                |
| Random Forest          | 0.7877 → 0.7932 (tuned) | Slight improvement after hyperparameter tuning; captures non-linear interactions |
| Gradient Boosting (GB) | CV mean: 0.8283         | Best performance after feature engineering; stable generalization across folds   |


### Inference: <br>

- Logistic Regression achieves 81.0%, showing that even a linear model captures significant
  predictive patterns. <br>
- Random Forest improves slightly with tuning (~79.3%), demonstrating the benefit of non-linear
  modeling. <br>
- Gradient Boosting achieves 82.8% average accuracy with cross-validation, reflecting its ability to   model complex relationships and generalize well.

## 2. Feature Insights

- Gender: Females had higher survival probability. <br>

- Passenger Class & Fare: Higher-class passengers and those paying higher fares had better survival. <br>

- Family Features: Passengers traveling alone (IsAlone) had lower survival; family size captures social/group effects.<br>

- Ticket-based Features: TicketFreq (group travel) and TicketPrefix (socio-economic proxy) further improve predictions.

### Inference:

- Survival is influenced by a combination of demographics, social grouping, and socio-economic status.

- Thoughtful feature engineering improves predictive accuracy, particularly for tree-based models like Random Forest and Gradient Boosting.

**Best model**: Gradient Boosting with engineered features achieves 82.8% cross-validated accuracy, providing a strong and reliable prediction.

Logistic Regression performs remarkably well (81.0%), indicating the dataset has clear linear trends in survival factors.